---
title: "Final Project 421"
author: "Josie Shaggy"
date: "December 5, 2018"
output:
  word_document: default
  html_document: default
---

```{r}
library(ggplot2)
library(lattice)
library(ranger)
library(caret)
```
#Introduction
My dataset is the "2017 Iditarod Trail Sled Dog Race" from Kaggle. In my project I am predicting whether or not a participant is a "Rookie" (first time participant) or a "Veteran" (a returning participant). This isn't necessarily the most useful target variable, but it would be helpful if someone were to bet on a participant, and are choosing between a rookie or veteran participant. There does not appear to be any other works on this problem.

There are 17 variables in this dataset:
  Number - Bib Number
  Name - Musher's Name
	Status - Status in the Race
	Country - Country of Origin
	Checkpoint - Checkpoint Name
	Latitude - location of checkpoint
	Longitude - location of checkpoint
	Distance - distance in miles from last checkpoint
	Time - time in hours from departure at the last checkpoint,
	Speed - average speed in miles per hour,
	Arrival Date
	Arrival Time
	Arrival Dogs - number of dogs at arrival
	Elapsed Time - layover time at the checkpoint
	Departure Date
	Departure Time
	Departure Dogs - Number of dogs at departure

I removed Arrival Time and Departure Time from the dataset because they had over 700 categories and they weren't useful in my models.

There are 1146 observations in this dataset.


#Missing Values Imputed by Hand
```{r}
dog = read.csv("report.csv")

dog$Distance[is.na(dog$Distance)] = mean(dog$Distance, na.rm = TRUE)
dog$Speed[is.na(dog$Speed)] = mean(dog$Speed, na.rm = TRUE)
dog$Arrival.Dogs[is.na(dog$Arrival.Dogs)] = mean(dog$Arrival.Dogs, na.rm = TRUE)
dog$Elapsed.Time[is.na(dog$Elapsed.Time)] = mean(dog$Elapsed.Time, na.rm = TRUE)
dog$Departure.Dogs[is.na(dog$Departure.Dogs)] = mean(dog$Departure.Dogs, na.rm = TRUE)
dog$Arrival.Time = NULL
dog$Departure.Time = NULL
```

#Recoding
```{r}
levels(dog$Name) = c("Male", "Female", "Male", "Female","Male", "Female","Male","Male","Male","Male","Female","Female","Male","Male","Male","Male","Male","Male","Female","Male","Male", "Female", "Male","Male","Male","Male","Female", "Female","Male","Male","Female","Female", "Male","Male","Female","Male","Male","Male","Male","Male","Male","Female","Male","Female","Female","Male","Female","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Female")

levels(dog$Checkpoint) = c("End", "Early", "Middle", "Middle", "End","End", "Middle", "Early", "Early","End", "Middle", "Middle","Early","End","Early","Middle","End")

levels(dog$Arrival.Date) = c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday","Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday")

levels(dog$Departure.Date) = c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday","Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday")

```


#Data Exploration / Visualization
```{r}
library(ggplot2)
ggplot(dog)+geom_bar(mapping = aes(x=Name, fill=Status), position="dodge")+ggtitle("Gender by Status")
```
This graph shows the gender and status of Iditarod participants.

```{r}
ggplot(dog)+geom_bar(mapping = aes(x=Country, fill=Status), position="dodge")+ggtitle("Status by Country")
```
This graph shows the country and status of Iditarod participants.

```{r}
ggplot(dog)+geom_bar(mapping = aes(x=Country, fill=Name), position="dodge")+ggtitle("Gender by Country")
```
This graph shows the country and gender of Iditarod participants.

```{r}
ggplot(dog)+geom_bar(mapping = aes(x=Checkpoint, fill=Status), position="dodge")+ggtitle("Checkpoint by Status")
```
This graph shows the status of the Iditarod participants who made it to each checkpoint.

```{r}
ggplot(dog)+geom_bar(mapping = aes(x=Arrival.Date, fill=Status), position="dodge")+ggtitle("Arrival Date by Status")
```

This graph shows the Arrival date and status of each Iditarod participant.
```{r}
ggplot(dog)+geom_bar(mapping = aes(x=Departure.Date, fill=Status), position="dodge")+ggtitle("Departure Date by Status")
```
This graph shows the departure date and status of each Iditarod participant.

```{r}
ggplot(dog)+geom_bar(mapping = aes(x=Country), position="dodge")+ggtitle("Count of Participants by Country")
```
This graph shows the number of participants in the Iditarod by country.

```{r}
ggplot(dog)+geom_bar(mapping = aes(x=Name), position="dodge")+ggtitle("Count of Gender")
```
This graph shows the number of participants in the Iditarod by gender.

```{r}
ggplot(dog)+geom_bar(mapping = aes(x=Status), position="dodge")+ggtitle("Graph of target variable")
```
This graph shows the Status of Iditarod participants.

```{r}
ggplot(dog)+geom_point(mapping = aes(x = Distance, y = Speed))+ggtitle("Graph of Speed by Distance")
```
This graph shows the speed and distance of the Iditarod participants.
```{r}
ggplot(dog)+geom_point(mapping = aes(x = Distance, y = Speed, color=Status))+ggtitle("Graph of Speed by Distance and Status")
```
This graph shows the distance and speed of Iditarod participants by their status. 

```{r}
ggplot(dog)+geom_point(mapping = aes(x = Distance, y = Speed, color=Country))+ggtitle("Graph of Speed by Distance and Country")
```
This graph shows the speed and distance of Iditarod participants by their country.

```{r}
ggplot(dog)+geom_density(mapping = aes(x = Time, color=Name))+ggtitle("Graph of Time and Gender")

```
This graph shows the density plot of time for Iditarod partcipants by gender.
```{r}
ggplot(dog)+geom_bar(mapping = aes(x=Checkpoint, fill=Country), position="dodge")+ggtitle("Checkpoint by Country")
```
This graph shows the distribution of participants from each country who made it to each checkpoint

```{r}
ggplot(dog)+geom_density(mapping = aes(x = Elapsed.Time, color=Checkpoint))+ggtitle("Graph of Elapsed Time and Gender")

```
This graph shows the density plot of Elapsed Time by checkpoint

###Proportion Table (Unbalanced)
```{r}
prop.table(table(dog$Status))

```



#Missing Values Imputed by Mean
```{r}
dogs = read.csv("report.csv")

dogs$Distance[is.na(dogs$Distance)] = mean(dogs$Distance, na.rm = TRUE)
dogs$Speed[is.na(dogs$Speed)] = mean(dogs$Speed, na.rm = TRUE)
dogs$Arrival.Dogs[is.na(dogs$Arrival.Dogs)] = mean(dogs$Arrival.Dogs, na.rm = TRUE)
dogs$Elapsed.Time[is.na(dogs$Elapsed.Time)] = mean(dogs$Elapsed.Time, na.rm = TRUE)
dogs$Departure.Dogs[is.na(dogs$Departure.Dogs)] = mean(dogs$Departure.Dogs, na.rm = TRUE)
dogs$Arrival.Time = NULL
dogs$Departure.Time = NULL

levels(dogs$Name) = c("Male", "Female", "Male", "Female","Male", "Female","Male","Male","Male","Male","Female","Female","Male","Male","Male","Male","Male","Male","Female","Male","Male", "Female", "Male","Male","Male","Male","Female", "Female","Male","Male","Female","Female", "Male","Male","Female","Male","Male","Male","Male","Male","Male","Female","Male","Female","Female","Male","Female","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Female")

levels(dogs$Checkpoint) = c("End", "Early", "Middle", "Middle", "End","End", "Middle", "Early", "Early","End", "Middle", "Middle","Early","End","Early","Middle","End")

levels(dogs$Arrival.Date) = c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday","Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday")

levels(dogs$Departure.Date) = c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday","Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday")

```

#Resampling - Mean Impute
```{r}
splitIndex <- createDataPartition(dogs$Status, p = .70, list = FALSE, times = 1)
train <- dogs[ splitIndex,]
test <- dogs[-splitIndex,]

train1 = train[train$Status =="Veteran",]
train0 = train[train$Status =="Rookie",]
n0 = nrow(train0)
n1 = nrow(train1)
train11 = train1[sample(1:n1, n0, replace = TRUE),]
train_over = rbind(train0, train11)
prop.table(table(train_over$Status))
```

#Random Forest - Mean Impute - No encoding
```{r}
library(caret)
library(lattice)
library(ggplot2)
library(ranger)

splitIndex <- createDataPartition(train_over$Status, p = .70, list = FALSE, times = 1)
train1 <- train_over[ splitIndex,]
test1 <- train_over[-splitIndex,]


model = ranger(Status ~., data = train1)
pred3 = predict(model, data = test1)$predictions
cm = confusionMatrix(pred3, test1$Status, positive ="Veteran")

cm$overall[1]
       
cm$byClass[11]
```

#model tuning for random forest - Mean Impute - no encoding
```{r}
myGrid = expand.grid(mtry = 3, splitrule = c("gini","extratrees"),
                     min.node.size = c(1:3))

model2 <- train(Status~.,data = train1, method = "ranger", 
               trControl = trainControl(method ="cv", number = 7, verboseIter = TRUE),
               tuneGrid = myGrid)
model2
```

#glmnet - Mean Impute - no encoding
```{r}
library(glmnet)

myControl = trainControl(
  method = "cv", number = 10)

model3 = train(Status~ ., train_over,
  method = "glmnet",
  trControl = myControl)

model3


```

#model tuning for glmnet - Mean Impute - no encoding
```{r}


model4 = train(
  Status ~ ., train_over,
  tuneGrid = expand.grid(alpha = 0:1,
  lambda = seq(0.0001, 1, length = 20)),
  method = "glmnet",
  trControl = myControl
)

model4

```


#tree - Mean Impute - no encoding
```{r}
library(rpart)
library(rattle)
mytree <- rpart(Status ~ ., data = train_over, method = "class")
mytree
fancyRpartPlot(mytree)
```


#model tuning for tree - Mean Impute - no encoding
```{r}
myGrid = expand.grid(mtry = 2, splitrule = c("gini","extratrees"),
                     min.node.size = c(1:3))

model5 <- train(Status~.,data = train_over, method = "rpart", 
               trControl = trainControl(method ="cv", number = 10, verboseIter = TRUE))

model5
```



#Encoding - Mean Impute - Dummy Variable
```{r}
dummies_model <- dummyVars(Status ~ ., data=dogs)

# Create the dummy variables using predict. The Y variable will not be present in trainData_mat.
trainData_mat <- predict(dummies_model, newdata = dogs)

# # Convert to dataframe
trainData <- data.frame(trainData_mat)
trainData$Status <- dogs$Status

prop.table(table(trainData$Status))

```


#Resampling - Mean Impute - dummy encoding
```{r}
splitIndex <- createDataPartition(trainData$Status, p = .70, list = FALSE, times = 1)
train <- trainData[ splitIndex,]
test <- trainData[-splitIndex,]

train1 = trainData[train$Status =="Veteran",]
train0 = trainData[train$Status =="Rookie",]
n0 = nrow(train0)
n1 = nrow(train1)
train11 = train1[sample(1:n1, n0, replace = TRUE),]
train_over1 = rbind(train0, train11)
prop.table(table(train_over1$Status))
```

#Random Forest - Mean Impute - dummy encoding
```{r}

splitIndex <- createDataPartition(train_over1$Status, p = .70, list = FALSE, times = 1)
train2 <- train_over1[ splitIndex,]
test2 <- train_over1[-splitIndex,]


model6 = ranger(Status ~., data = train2)
pred1 = predict(model6, data = test2)$predictions
cm1 = confusionMatrix(pred1, test2$Status, positive ="Veteran")

cm1$overall[1]
       
cm1$byClass[11]
```

#model tuning for random forest - Mean Impute - dummy encoding
```{r}
myGrid = expand.grid(mtry = 3, splitrule = c("gini","extratrees"),
                     min.node.size = c(1:3))

model7 <- train(Status~.,data = train2, method = "ranger", 
               trControl = trainControl(method ="cv", number = 7, verboseIter = TRUE),
               tuneGrid = myGrid)
model7
```

#glmnet - Mean Impute - dummy encoding
```{r}
library(glmnet)

myControl = trainControl(
  method = "cv", number = 10)

model8 = train(Status~ ., train_over1,
  method = "glmnet",
  trControl = myControl)

model8

```

#model tuning for glmnet - Mean Impute - dummy encoding
```{r}
model9 = train(
  Status ~ ., train_over1,
  tuneGrid = expand.grid(alpha = 0:1,
  lambda = seq(0.0001, 1, length = 20)),
  method = "glmnet",
  trControl = myControl
)

model9

```


#tree - Mean Impute - dummy encoding
```{r}
library(rpart)
library(rattle)
mytree1 <- rpart(Status ~ ., data = train_over1, method = "class")
mytree1
fancyRpartPlot(mytree1)
```


#model tuning for tree - Mean Impute - dummy encoding
```{r}
myGrid = expand.grid(mtry = 2, splitrule = c("gini","extratrees"),
                     min.node.size = c(1:3))

model10 <- train(Status~.,data = train_over1, method = "rpart", 
               trControl = trainControl(method ="cv", number = 10, verboseIter = TRUE))

model10
```



#Encoding with all numerics - Mean Impute
```{r}
d = dogs
levels(d$Name) = c("Male", "Female", "Male", "Female","Male", "Female","Male","Male","Male","Male","Female","Female","Male","Male","Male","Male","Male","Male","Female","Male","Male", "Female", "Male","Male","Male","Male","Female", "Female","Male","Male","Female","Female", "Male","Male","Female","Male","Male","Male","Male","Male","Male","Female","Male","Female","Female","Male","Female","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Female")

levels(d$Checkpoint) = c("End", "Early", "Middle", "Middle", "End","End", "Middle", "Early", "Early","End", "Middle", "Middle","Early","End","Early","Middle","End")

levels(d$Arrival.Date) = c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday","Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday")

levels(d$Departure.Date) = c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday","Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday")
d$Distance[is.na(dogs$Distance)] = mean(dogs$Distance, na.rm = TRUE)
d$Speed[is.na(dogs$Speed)] = mean(dogs$Speed, na.rm = TRUE)
d$Arrival.Dogs[is.na(dogs$Arrival.Dogs)] = mean(dogs$Arrival.Dogs, na.rm = TRUE)
d$Elapsed.Time[is.na(dogs$Elapsed.Time)] = mean(dogs$Elapsed.Time, na.rm = TRUE)
d$Departure.Dogs[is.na(dogs$Departure.Dogs)] = mean(dogs$Departure.Dogs, na.rm = TRUE)
d$Arrival.Time = NULL
d$Departure.Time = NULL
d$Name = as.numeric(d$Name)
d$Country = as.numeric(d$Country)
d$Checkpoint = as.numeric(d$Checkpoint)
d$Arrival.Date = as.numeric(d$Arrival.Date)
d$Departure.Date = as.numeric(d$Departure.Date)


```

#Resampling - Mean Impute - numeric encoding
```{r}
splitIndex <- createDataPartition(d$Status, p = .70, list = FALSE, times = 1)
train <- d[ splitIndex,]
test <- d[-splitIndex,]

train1 = train[train$Status =="Veteran",]
train0 = train[train$Status =="Rookie",]
n0 = nrow(train0)
n1 = nrow(train1)
train11 = train[sample(1:n1, n0, replace = TRUE),]
train_over2 = rbind(train0, train11)
prop.table(table(train_over2$Status))
```

#Random Forest - Mean Impute - numeric encoding
```{r}
library(caret)
library(lattice)
library(ggplot2)
library(ranger)

splitIndex <- createDataPartition(train_over2$Status, p = .70, list = FALSE, times = 1)
train3 <- train_over2[ splitIndex,]
test3 <- train_over2[-splitIndex,]


model11 = ranger(Status ~., data = train3)
pred2 = predict(model11, data = test3)$predictions
cm2 = confusionMatrix(pred2, test3$Status, positive ="Veteran")

cm2$overall[1]
       
cm2$byClass[11]
```

#model tuning for random forest - Mean Impute - numeric encoding
```{r}
myGrid = expand.grid(mtry = 3, splitrule = c("gini","extratrees"),
                     min.node.size = c(1:3))

model12 <- train(Status~.,data = train3, method = "ranger", 
               trControl = trainControl(method ="cv", number = 7, verboseIter = TRUE),
               tuneGrid = myGrid)
model12
```

#glmnet - Mean Impute - numeric encoding
```{r}
library(glmnet)

myControl = trainControl(
  method = "cv", number = 10)

model13 = train(Status~ ., train_over2,
  method = "glmnet",
  trControl = myControl)

model13


```

#model tuning for glmnet - Mean Impute - numeric encoding
```{r}

model14 = train(
  Status ~ ., train_over2,
  tuneGrid = expand.grid(alpha = 0:1,
  lambda = seq(0.0001, 1, length = 20)),
  method = "glmnet",
  trControl = myControl
)

model14

```


#tree - Mean Impute - numeric encoding
```{r}
library(rpart)
library(rattle)
mytree2 <- rpart(Status ~ ., data = train_over2, method = "class")
mytree2
fancyRpartPlot(mytree2)
```


#model tuning for tree - Mean Impute - numeric encoding
```{r}
myGrid = expand.grid(mtry = 2, splitrule = c("gini","extratrees"),
                     min.node.size = c(1:3))

model15 <- train(Status~.,data = train_over2, method = "rpart", 
               trControl = trainControl(method ="cv", number = 10, verboseIter = TRUE))

model15
```


#Missing Values Imputed by MedianImpute
```{r}
dogs = read.csv("report.csv")
dogs1 <- preProcess(dogs, method='medianImpute')

# Predict generate the imputed data 
dogs1<- predict(dogs1, newdata = dogs)

dogs1$Arrival.Time = NULL
dogs1$Departure.Time = NULL

levels(dogs1$Name) = c("Male", "Female", "Male", "Female","Male", "Female","Male","Male","Male","Male","Female","Female","Male","Male","Male","Male","Male","Male","Female","Male","Male", "Female", "Male","Male","Male","Male","Female", "Female","Male","Male","Female","Female", "Male","Male","Female","Male","Male","Male","Male","Male","Male","Female","Male","Female","Female","Male","Female","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Female")

levels(dogs1$Checkpoint) = c("End", "Early", "Middle", "Middle", "End","End", "Middle", "Early", "Early","End", "Middle", "Middle","Early","End","Early","Middle","End")

levels(dogs1$Arrival.Date) = c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday","Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday")

levels(dogs1$Departure.Date) = c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday","Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday")

```

#Resampling - Median Impute - no encoding
```{r}
splitIndex <- createDataPartition(dogs1$Status, p = .70, list = FALSE, times = 1)
train <- dogs1[ splitIndex,]
test <- dogs1[-splitIndex,]

train1 = train[train$Status =="Veteran",]
train0 = train[train$Status =="Rookie",]
n0 = nrow(train0)
n1 = nrow(train1)
train11 = train1[sample(1:n1, n0, replace = TRUE),]
train_over3 = rbind(train0, train11)
prop.table(table(train_over3$Status))
```

#Random Forest - Median Impute - no encoding 
```{r}
library(caret)
library(lattice)
library(ggplot2)
library(ranger)

splitIndex <- createDataPartition(train_over3$Status, p = .70, list = FALSE, times = 1)
train4 <- train_over3[ splitIndex,]
test4 <- train_over3[-splitIndex,]


model16 = ranger(Status ~., data = train4)
pred4 = predict(model16, data = test4)$predictions
cm3 = confusionMatrix(pred4, test4$Status, positive ="Veteran")

cm3$overall[1]
       
cm3$byClass[11]
```

#model tuning for random forest - Median Impute - no encoding
```{r}
myGrid = expand.grid(mtry = 3, splitrule = c("gini","extratrees"),
                     min.node.size = c(1:3))

model17 <- train(Status~.,data = train4, method = "ranger", 
               trControl = trainControl(method ="cv", number = 7, verboseIter = TRUE),
               tuneGrid = myGrid)
model17
```

#glmnet - Median Impute - no encoding
```{r}
library(glmnet)

myControl = trainControl(
  method = "cv", number = 10)

model18 = train(Status~ ., train_over3,
  method = "glmnet",
  trControl = myControl)

model18


```

#model tuning for glmnet - Median Impute - no encoding
```{r}


model19 = train(
  Status ~ ., train_over3,
  tuneGrid = expand.grid(alpha = 0:1,
  lambda = seq(0.0001, 1, length = 20)),
  method = "glmnet",
  trControl = myControl
)

model19

```


#tree - Median Impute - no encoding
```{r}
library(rpart)
library(rattle)
mytree3 <- rpart(Status ~ ., data = train_over3, method = "class")
mytree3
fancyRpartPlot(mytree3)
```


#model tuning for tree - Median Impute - no encoding
```{r}
myGrid = expand.grid(mtry = 2, splitrule = c("gini","extratrees"),
                     min.node.size = c(1:3))

model20 <- train(Status~.,data = train_over3, method = "rpart", 
               trControl = trainControl(method ="cv", number = 10, verboseIter = TRUE))

model20
```


#Encoding - Dummy Variable - Median Impute
```{r}
dummies_model1 <- dummyVars(Status ~ ., data=dogs1)

# Create the dummy variables using predict. The Y variable will not be present in trainData_mat.
trainData_mat2 <- predict(dummies_model1, newdata = dogs1)

# # Convert to dataframe
trainData2 <- data.frame(trainData_mat2)
trainData2$Status <- dogs1$Status

prop.table(table(trainData2$Status))

```


#Resampling - Median Impute - dummy encoding
```{r}
splitIndex <- createDataPartition(trainData2$Status, p = .70, list = FALSE, times = 1)
train <- trainData2[ splitIndex,]
test <- trainData2[-splitIndex,]

train1 = trainData2[train$Status =="Veteran",]
train0 = trainData2[train$Status =="Rookie",]
n0 = nrow(train0)
n1 = nrow(train1)
train11 = train1[sample(1:n1, n0, replace = TRUE),]
train_over4 = rbind(train0, train11)

prop.table(table(train_over4$Status))
```

#Random Forest - Median Impute - dummy encoding
```{r}
library(caret)
library(lattice)
library(ggplot2)
library(ranger)

splitIndex <- createDataPartition(train_over4$Status, p = .70, list = FALSE, times = 1)
train5 <- train_over4[ splitIndex,]
test5 <- train_over4[-splitIndex,]


model21 = ranger(Status ~., data = train5)
pred5 = predict(model21, data = test5)$predictions
cm4 = confusionMatrix(pred5, test5$Status, positive ="Veteran")

cm4$overall[1]
       
cm4$byClass[11]
```

#model tuning for random forest - Median Impute - dummy encoding
```{r}
myGrid = expand.grid(mtry = 3, splitrule = c("gini","extratrees"),
                     min.node.size = c(1:3))

model22 <- train(Status~.,data = train5, method = "ranger", 
               trControl = trainControl(method ="cv", number = 7, verboseIter = TRUE),
               tuneGrid = myGrid)
model22
```

#glmnet - Median Impute - dummy encoding
```{r}
library(glmnet)

myControl = trainControl(
  method = "cv", number = 10)

model23 = train(Status~ ., train_over4,
  method = "glmnet",
  trControl = myControl)

model23

```

#model tuning for glmnet - Median Impute - dummy encoding
```{r}


model24 = train(
  Status ~ ., train_over4,
  tuneGrid = expand.grid(alpha = 0:1,
  lambda = seq(0.0001, 1, length = 20)),
  method = "glmnet",
  trControl = myControl
)

model24

```


#tree - Median Impute - dummy encoding
```{r}
library(rpart)
library(rattle)
mytree4 <- rpart(Status ~ ., data = train_over4, method = "class")
mytree4
fancyRpartPlot(mytree4)
```


#model tuning for tree - Median Impute - dummy encoding
```{r}
myGrid = expand.grid(mtry = 2, splitrule = c("gini","extratrees"),
                     min.node.size = c(1:3))

model25 <- train(Status~.,data = train_over4, method = "rpart", 
               trControl = trainControl(method ="cv", number = 10, verboseIter = TRUE))

model25
```


#Encoding with all numerics
```{r}
dogs2 <- preProcess(dogs, method='medianImpute')

# Predict generate the imputed data 
dogs2<- predict(dogs2, newdata = dogs)

dogs2$Arrival.Time = NULL
dogs2$Departure.Time = NULL

levels(dogs2$Name) = c("Male", "Female", "Male", "Female","Male", "Female","Male","Male","Male","Male","Female","Female","Male","Male","Male","Male","Male","Male","Female","Male","Male", "Female", "Male","Male","Male","Male","Female", "Female","Male","Male","Female","Female", "Male","Male","Female","Male","Male","Male","Male","Male","Male","Female","Male","Female","Female","Male","Female","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Female")

levels(dogs2$Checkpoint) = c("End", "Early", "Middle", "Middle", "End","End", "Middle", "Early", "Early","End", "Middle", "Middle","Early","End","Early","Middle","End")

levels(dogs2$Arrival.Date) = c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday","Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday")

levels(dogs2$Departure.Date) = c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday","Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday")
d1 = dogs2
d1$Arrival.Time = NULL
d1$Departure.Time = NULL
d1$Arrival.Time = NULL
d1$Departure.Time = NULL
d1$Name = as.numeric(d1$Name)
d1$Country = as.numeric(d1$Country)
d1$Checkpoint = as.numeric(d1$Checkpoint)
d1$Arrival.Date = as.numeric(d1$Arrival.Date)
d1$Departure.Date = as.numeric(d1$Departure.Date)


```

#Resampling - Median Impute -numeric encoding
```{r}
splitIndex <- createDataPartition(d1$Status, p = .70, list = FALSE, times = 1)
train <- d1[ splitIndex,]
test <- d1[-splitIndex,]

train1 = train[train$Status =="Veteran",]
train0 = train[train$Status =="Rookie",]
n0 = nrow(train0)
n1 = nrow(train1)
train11 = train[sample(1:n1, n0, replace = TRUE),]
train_over5 = rbind(train0, train11)
prop.table(table(train_over5$Status))
```

#Random Forest - Median Impute -numeric encoding
```{r}
library(caret)
library(lattice)
library(ggplot2)
library(ranger)

splitIndex <- createDataPartition(train_over5$Status, p = .70, list = FALSE, times = 1)
train6 <- train_over5[ splitIndex,]
test6 <- train_over5[-splitIndex,]


model26 = ranger(Status ~., data = train6)
pred6 = predict(model26, data = test6)$predictions
cm5 = confusionMatrix(pred6, test6$Status, positive ="Veteran")

cm5$overall[1]
       
cm5$byClass[11]
```

#model tuning for random forest - Median Impute -numeric encoding
```{r}
myGrid = expand.grid(mtry = 3, splitrule = c("gini","extratrees"),
                     min.node.size = c(1:3))

model27 <- train(Status~.,data = train6, method = "ranger", 
               trControl = trainControl(method ="cv", number = 7, verboseIter = TRUE),
               tuneGrid = myGrid)
model27
```

#glmnet - Median Impute - numeric encoding
```{r}
library(glmnet)

myControl = trainControl(
  method = "cv", number = 10)

model28 = train(Status~ ., train_over5,
  method = "glmnet",
  trControl = myControl)

model28


```

#model tuning for glmnet - Median Impute - numeric encoding
```{r}



model29 = train(
  Status ~ ., train_over5,
  tuneGrid = expand.grid(alpha = 0:1,
  lambda = seq(0.0001, 1, length = 20)),
  method = "glmnet",
  trControl = myControl
)

model29

```


#tree - Median Impute -numeric encoding
```{r}
library(rpart)
library(rattle)
mytree5 <- rpart(Status ~ ., data = train_over5, method = "class")
mytree5
fancyRpartPlot(mytree5)
```


#model tuning for tree - Median Impute - numeric encoding
```{r}
myGrid = expand.grid(mtry = 2, splitrule = c("gini","extratrees"),
                     min.node.size = c(1:3))

model30 <- train(Status~.,data = train_over5, method = "rpart", 
               trControl = trainControl(method ="cv", number = 10, verboseIter = TRUE))

model30
```


#Missing Values using knnImpute
```{r}
dogs = read.csv("report.csv")

dogs3 <- preProcess(dogs, method='knnImpute')

# Predict generate the imputed data 
dogs3<- predict(dogs3, newdata = dogs)

dogs3$Arrival.Time = NULL
dogs3$Departure.Time = NULL

levels(dogs3$Name) = c("Male", "Female", "Male", "Female","Male", "Female","Male","Male","Male","Male","Female","Female","Male","Male","Male","Male","Male","Male","Female","Male","Male", "Female", "Male","Male","Male","Male","Female", "Female","Male","Male","Female","Female", "Male","Male","Female","Male","Male","Male","Male","Male","Male","Female","Male","Female","Female","Male","Female","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Female")

levels(dogs3$Checkpoint) = c("End", "Early", "Middle", "Middle", "End","End", "Middle", "Early", "Early","End", "Middle", "Middle","Early","End","Early","Middle","End")

levels(dogs3$Arrival.Date) = c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday","Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday")

levels(dogs3$Departure.Date) = c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday","Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday")

```

#Resampling - KNNImpute - no encoding
```{r}
splitIndex <- createDataPartition(dogs3$Status, p = .70, list = FALSE, times = 1)
train <- dogs3[ splitIndex,]
test <- dogs3[-splitIndex,]

train1 = train[train$Status =="Veteran",]
train0 = train[train$Status =="Rookie",]
n0 = nrow(train0)
n1 = nrow(train1)
train11 = train1[sample(1:n1, n0, replace = TRUE),]
train_over6 = rbind(train0, train11)
prop.table(table(train_over6$Status))
```

#Random Forest - KNNImpute - no encoding
```{r}
library(caret)
library(lattice)
library(ggplot2)
library(ranger)

splitIndex <- createDataPartition(train_over6$Status, p = .70, list = FALSE, times = 1)
train7 <- train_over6[ splitIndex,]
test7 <- train_over6[-splitIndex,]


model31 = ranger(Status ~., data = train7)
pred7 = predict(model31, data = test7)$predictions
cm6 = confusionMatrix(pred7, test7$Status, positive ="Veteran")

cm6$overall[1]
       
cm6$byClass[11]
```

#model tuning for random forest - KNNImpute - no encoding
```{r}
myGrid = expand.grid(mtry = 3, splitrule = c("gini","extratrees"),
                     min.node.size = c(1:3))

model32 <- train(Status~.,data = train7, method = "ranger", 
               trControl = trainControl(method ="cv", number = 7, verboseIter = TRUE),
               tuneGrid = myGrid)
model32
```

#glmnet - KNNImpute - no encoding
```{r}
library(glmnet)

myControl = trainControl(
  method = "cv", number = 10)

model33 = train(Status~ ., train_over6,
  method = "glmnet",
  trControl = myControl)

model33


```

#model tuning for glmnet - KNNImpute - no encoding
```{r}


model34 = train(
  Status ~ ., train_over6,
  tuneGrid = expand.grid(alpha = 0:1,
  lambda = seq(0.0001, 1, length = 20)),
  method = "glmnet",
  trControl = myControl
)

model34

```


#tree - KNNImpute - no encoding
```{r}
library(rpart)
library(rattle)
mytree6 <- rpart(Status ~ ., data = train_over6, method = "class")
mytree6
fancyRpartPlot(mytree6)
```


#model tuning for tree - KNNImpute - no encoding
```{r}
myGrid = expand.grid(mtry = 2, splitrule = c("gini","extratrees"),
                     min.node.size = c(1:3))

model35 <- train(Status~.,data = train_over6, method = "rpart", 
               trControl = trainControl(method ="cv", number = 10, verboseIter = TRUE))

model35
```


#Encoding - Dummy Variable
```{r}
dummies_model3 <- dummyVars(Status ~ ., data=dogs3)

# Create the dummy variables using predict. The Y variable will not be present in trainData_mat.
trainData_mat3 <- predict(dummies_model3, newdata = dogs3)

# # Convert to dataframe
trainData3 <- data.frame(trainData_mat3)
trainData3$Status <- dogs3$Status
prop.table(table(trainData3$Status))

```


#Resampling - KNNImpute - dummy encoding
```{r}
splitIndex <- createDataPartition(trainData3$Status, p = .70, list = FALSE, times = 1)
train <- trainData3[ splitIndex,]
test <- trainData3[-splitIndex,]

train1 = trainData3[train$Status =="Veteran",]
train0 = trainData3[train$Status =="Rookie",]
n0 = nrow(train0)
n1 = nrow(train1)
train11 = train1[sample(1:n1, n0, replace = TRUE),]
train_over7 = rbind(train0, train11)
prop.table(table(train_over7$Status))
```

#Random Forest - KNNImpute - dummy encoding
```{r}
library(caret)
library(lattice)
library(ggplot2)
library(ranger)

splitIndex <- createDataPartition(train_over7$Status, p = .70, list = FALSE, times = 1)
train8 <- train_over7[ splitIndex,]
test8 <- train_over7[-splitIndex,]


model36 = ranger(Status ~., data = train8)
pred8 = predict(model36, data = test8)$predictions
cm7 = confusionMatrix(pred8, test8$Status, positive ="Veteran")

cm7$overall[1]
       
cm7$byClass[11]
```

#model tuning for random forest - KNNImpute - dummy encoding
```{r}
myGrid = expand.grid(mtry = 3, splitrule = c("gini","extratrees"),
                     min.node.size = c(1:3))

model37 <- train(Status~.,data = train8, method = "ranger", 
               trControl = trainControl(method ="cv", number = 7, verboseIter = TRUE),
               tuneGrid = myGrid)
model37
```

#glmnet - KNNImpute - dummy encoding
```{r}
library(glmnet)

myControl = trainControl(
  method = "cv", number = 10)

model38 = train(Status~ ., train_over7,
  method = "glmnet",
  trControl = myControl)

model38

```

#model tuning for glmnet - KNNImpute - dummy encoding
```{r}


model39 = train(
  Status ~ ., train_over7,
  tuneGrid = expand.grid(alpha = 0:1,
  lambda = seq(0.0001, 1, length = 20)),
  method = "glmnet",
  trControl = myControl
)

model39

```


#tree - KNNImpute - dummy encoding
```{r}
library(rpart)
library(rattle)
mytree7 <- rpart(Status ~ ., data = train_over7, method = "class")
mytree7
fancyRpartPlot(mytree7)
```


#model tuning for tree - KNNImpute - dummy encoding
```{r}
myGrid = expand.grid(mtry = 2, splitrule = c("gini","extratrees"),
                     min.node.size = c(1:3))

model40 <- train(Status~.,data = train_over7, method = "rpart", 
               trControl = trainControl(method ="cv", number = 10, verboseIter = TRUE))

model40
```


#Encoding with all numerics
```{r}
dogs4 <- preProcess(dogs, method='knnImpute')

# Predict generate the imputed data 
dogs4<- predict(dogs4, newdata = dogs)

dogs4$Arrival.Time = NULL
dogs4$Departure.Time = NULL

levels(dogs4$Name) = c("Male", "Female", "Male", "Female","Male", "Female","Male","Male","Male","Male","Female","Female","Male","Male","Male","Male","Male","Male","Female","Male","Male", "Female", "Male","Male","Male","Male","Female", "Female","Male","Male","Female","Female", "Male","Male","Female","Male","Male","Male","Male","Male","Male","Female","Male","Female","Female","Male","Female","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Male","Female")

levels(dogs4$Checkpoint) = c("End", "Early", "Middle", "Middle", "End","End", "Middle", "Early", "Early","End", "Middle", "Middle","Early","End","Early","Middle","End")

levels(dogs4$Arrival.Date) = c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday","Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday")

levels(dogs4$Departure.Date) = c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday","Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday")
d2 = dogs4
d2$Arrival.Time = NULL
d2$Departure.Time = NULL
d2$Arrival.Time = NULL
d2$Departure.Time = NULL
d2$Name = as.numeric(d2$Name)
d2$Country = as.numeric(d2$Country)
d2$Checkpoint = as.numeric(d2$Checkpoint)
d2$Arrival.Date = as.numeric(d2$Arrival.Date)
d2$Departure.Date = as.numeric(d2$Departure.Date)


```

#Resampling - KNNImpute - numeric encoding
```{r}
splitIndex <- createDataPartition(d2$Status, p = .70, list = FALSE, times = 1)
train <- d2[ splitIndex,]
test <- d2[-splitIndex,]

train1 = train[train$Status =="Veteran",]
train0 = train[train$Status =="Rookie",]
n0 = nrow(train0)
n1 = nrow(train1)
train11 = train[sample(1:n1, n0, replace = TRUE),]
train_over8 = rbind(train0, train11)
prop.table(table(train_over8$Status))
```

#Random Forest - KNNImpute - numeric encoding
```{r}
library(caret)
library(lattice)
library(ggplot2)
library(ranger)

splitIndex <- createDataPartition(train_over8$Status, p = .70, list = FALSE, times = 1)
train9 <- train_over8[ splitIndex,]
test9 <- train_over8[-splitIndex,]


model41 = ranger(Status ~., data = train9)
pred9 = predict(model41, data = test9)$predictions
cm8 = confusionMatrix(pred9, test9$Status, positive ="Veteran")

cm8$overall[1]
       
cm8$byClass[11]
```

#model tuning for random forest - KNNImpute - numeric encoding
```{r}
myGrid = expand.grid(mtry = 3, splitrule = c("gini","extratrees"),
                     min.node.size = c(1:3))

model42 <- train(Status~.,data = train9, method = "ranger", 
               trControl = trainControl(method ="cv", number = 7, verboseIter = TRUE),
               tuneGrid = myGrid)
model42
```

#glmnet - KNNImpute - numeric encoding
```{r}
library(glmnet)

myControl = trainControl(
  method = "cv", number = 10)

model43 = train(Status~ ., train_over8,
  method = "glmnet",
  trControl = myControl)

model43


```

#model tuning for glmnet - KNNImpute - numeric encoding
```{r}



model44 = train(
  Status ~ ., train_over8,
  tuneGrid = expand.grid(alpha = 0:1,
  lambda = seq(0.0001, 1, length = 20)),
  method = "glmnet",
  trControl = myControl
)

model44

```


#tree - KNNImpute - numeric encoding
```{r}
library(rpart)
library(rattle)
mytree8 <- rpart(Status ~ ., data = train_over8, method = "class")
mytree8
fancyRpartPlot(mytree8)
```


#model tuning for tree - KNNImpute - numeric encoding
```{r}
myGrid = expand.grid(mtry = 2, splitrule = c("gini","extratrees"),
                     min.node.size = c(1:3))

model45 <- train(Status~.,data = train_over8, method = "rpart", 
               trControl = trainControl(method ="cv", number = 10, verboseIter = TRUE))

model45
```

#Conclusion
My final model is the Random Forest using Mean Impute with Dummy Encoding. The Accuracy was 0.0021362 below the Random Forest using KNNImpute wit Numeric Encoding, but the Balanced Accruracy was 0.0129426 higher. Because the Balanced Accuracy was higher, I chose the Random Forest using Mean Impute with Dummy Encoding. At the time I was typing this, the Accuracy for RF with Mean Impute and Dummy Encoding was 0.8857143 and the Balanced Accuracry was 0.8850786.

##Random Forest - Mean Impute - dummy encoding
```{r}
cm1$overall[1]
       
cm1$byClass[11]
```

##Random Forest - KNNImpute - numeric encoding
```{r}
cm8$overall[1]
       
cm8$byClass[11]
```
